In [1]:
import os
import nibabel as nib
import numpy as np

In [2]:
Dir = r"C:\Users\Rishabh\Downloads\label_192\label_192"
ground_truths = os.path.join(Dir, 'ground truths')
images = os.path.join(Dir, 'images')

In [3]:
patch_shape = [64, 64, 16]
stride_shape = [16, 16, 8]

In [4]:
index = 10
nii_segementation = nib.load(os.path.join(ground_truths, os.listdir(images)[index]))
nii_image = nib.load(os.path.join(images, os.listdir(images)[index]))

# Get the image data as a NumPy array
image_data = nii_image.get_fdata()
segementation_data = nii_segementation.get_fdata()
print(np.unique(segementation_data))
image_data.shape, segementation_data.shape

[0. 1.]


((512, 512, 32), (512, 512, 32))

In [5]:
os.path.join(images, os.listdir(images)[index])

'C:\\Users\\Rishabh\\Downloads\\label_192\\label_192\\images\\ID_0b10cbee_ID_f91d6a7cd2.nii.gz'

In [6]:
volume_index = [0,0,0]
count = 0
while True:
    count = count + 1
    print(count, ':-',volume_index)  
    if volume_index[0] + patch_shape[0] < image_data.shape[0]:
        volume_index[0] = volume_index[0] + stride_shape[0]
        
    elif volume_index[1] + patch_shape[1] < image_data.shape[1]:
        volume_index[0] = 0
        volume_index[1] = volume_index[1] + stride_shape[1]
        
    elif volume_index[2] + patch_shape[2] < image_data.shape[2]:
        volume_index[0] = 0
        volume_index[1] = 0
        volume_index[2] = volume_index[2] + stride_shape[2]
    
    else:
        break
print(count)

1 :- [0, 0, 0]
2 :- [16, 0, 0]
3 :- [32, 0, 0]
4 :- [48, 0, 0]
5 :- [64, 0, 0]
6 :- [80, 0, 0]
7 :- [96, 0, 0]
8 :- [112, 0, 0]
9 :- [128, 0, 0]
10 :- [144, 0, 0]
11 :- [160, 0, 0]
12 :- [176, 0, 0]
13 :- [192, 0, 0]
14 :- [208, 0, 0]
15 :- [224, 0, 0]
16 :- [240, 0, 0]
17 :- [256, 0, 0]
18 :- [272, 0, 0]
19 :- [288, 0, 0]
20 :- [304, 0, 0]
21 :- [320, 0, 0]
22 :- [336, 0, 0]
23 :- [352, 0, 0]
24 :- [368, 0, 0]
25 :- [384, 0, 0]
26 :- [400, 0, 0]
27 :- [416, 0, 0]
28 :- [432, 0, 0]
29 :- [448, 0, 0]
30 :- [0, 16, 0]
31 :- [16, 16, 0]
32 :- [32, 16, 0]
33 :- [48, 16, 0]
34 :- [64, 16, 0]
35 :- [80, 16, 0]
36 :- [96, 16, 0]
37 :- [112, 16, 0]
38 :- [128, 16, 0]
39 :- [144, 16, 0]
40 :- [160, 16, 0]
41 :- [176, 16, 0]
42 :- [192, 16, 0]
43 :- [208, 16, 0]
44 :- [224, 16, 0]
45 :- [240, 16, 0]
46 :- [256, 16, 0]
47 :- [272, 16, 0]
48 :- [288, 16, 0]
49 :- [304, 16, 0]
50 :- [320, 16, 0]
51 :- [336, 16, 0]
52 :- [352, 16, 0]
53 :- [368, 16, 0]
54 :- [384, 16, 0]
55 :- [400, 16, 0]
56 :- [41

In [7]:
volume_index = [352, 464, 24]
input_volume = image_data[volume_index[0]:volume_index[0]+patch_shape[0], volume_index[1]:volume_index[1]+patch_shape[1], volume_index[2]:volume_index[2]+patch_shape[2]]

In [8]:
# input_volume.shape
# for i in range(3):
#     empty_volume = 
#     print(input_volume.shape[i])
print(input_volume.shape)
if input_volume.shape[0] != patch_shape[0]:
    empty_volume = np.zeros((patch_shape[0]-input_volume.shape[0], input_volume.shape[1], input_volume.shape[2]))
    input_volume = np.concatenate((input_volume, empty_volume), axis=0)

if input_volume.shape[1] != patch_shape[1]:
    empty_volume = np.zeros((input_volume.shape[0], patch_shape[1]-input_volume.shape[1], input_volume.shape[2]))
    input_volume = np.concatenate((input_volume, empty_volume), axis=1)


if input_volume.shape[2] != patch_shape[2]:
    empty_volume = np.zeros((input_volume.shape[0], input_volume.shape[1], patch_shape[2]-input_volume.shape[2]))
    input_volume = np.concatenate((input_volume, empty_volume), axis=2)

print(input_volume.shape)

(64, 48, 8)
(64, 64, 16)


In [246]:
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

class Single_Volume_patch_Class_3D(Dataset):
    def __init__(self, ImagePath, MaskPath, transform=None, patch_shape = [64, 64, 16] , stride_shape = [16, 16, 8]):
        self.ImagePath = ImagePath
        self.MaskPath = MaskPath
        nii_segementation = nib.load(self.MaskPath)
        nii_image = nib.load(self.ImagePath)
        image_data = nii_image.get_fdata()
        segementation_data = nii_segementation.get_fdata()
        self.Height = image_data.shape[0] 
        self.Width = image_data.shape[1]
        self.depth = image_data.shape[2]
        self.patch_shape = patch_shape
        self.stride_shape = stride_shape
        self.transform = transform


    def get_image_and_mask(self, volume_index):
        nii_segementation = nib.load(self.MaskPath)
        nii_image = nib.load(self.ImagePath)
        image_data = nii_image.get_fdata()
        segementation_data = nii_segementation.get_fdata()
        input_volume = image_data[volume_index[0]:volume_index[0]+self.patch_shape[0], volume_index[1]:volume_index[1]+self.patch_shape[1], volume_index[2]:volume_index[2]+self.patch_shape[2]]
        segementation_data = segementation_data[volume_index[0]:volume_index[0]+self.patch_shape[0], volume_index[1]:volume_index[1]+self.patch_shape[1], volume_index[2]:volume_index[2]+self.patch_shape[2]]

        if input_volume.shape[0] != patch_shape[0]:
            empty_volume = np.zeros((patch_shape[0]-input_volume.shape[0], input_volume.shape[1], input_volume.shape[2]))
            input_volume = np.concatenate((input_volume, empty_volume), axis=0)
            segementation_data = np.concatenate((segementation_data, empty_volume), axis=0)
        
        if input_volume.shape[1] != patch_shape[1]:
            empty_volume = np.zeros((input_volume.shape[0], patch_shape[1]-input_volume.shape[1], input_volume.shape[2]))
            input_volume = np.concatenate((input_volume, empty_volume), axis=1)
            segementation_data = np.concatenate((segementation_data, empty_volume), axis=1)
        
        
        if input_volume.shape[2] != patch_shape[2]:
            empty_volume = np.zeros((input_volume.shape[0], input_volume.shape[1], patch_shape[2]-input_volume.shape[2]))
            input_volume = np.concatenate((input_volume, empty_volume), axis=2)
            segementation_data = np.concatenate((segementation_data, empty_volume), axis=2)

        return input_volume, segementation_data

    def transform_volume(self, image_volume, mask_volume):
        transformed = self.transform(
                image=image_volume, 
                mask=mask_volume
            )
        images = transformed['image']
        masks = transformed['mask'].permute(2, 0, 1)
        return images , masks
        
        
    def __len__(self):
        self.x_patches = (self.Height-self.patch_shape[0])//self.stride_shape[0]+1
        self.y_patches = (self.Width-self.patch_shape[1])//self.stride_shape[1]+1
        self.z_patches = (self.depth-self.patch_shape[2])//self.stride_shape[2]+1
        self.num_patches = self.x_patches*self.y_patches*self.z_patches
        return self.num_patches

        
        
    def __getitem__(self, index):
        slice_number = (index//(self.x_patches*self.y_patches))*self.stride_shape[2]
        index = index%(self.x_patches*self.y_patches)
        row_number = (index//self.y_patches)*self.stride_shape[1]
        index = (index%self.y_patches)*self.stride_shape[0]
        x, y, z = index , row_number, slice_number
        volume_index = [x, y, z]
        image_volume , mask_volume = self.get_image_and_mask(volume_index)

        if self.transform is not None:
            transformed_image_volume, transformed_mask_volume = self.transform_volume(image_volume, mask_volume)

        return transformed_image_volume, transformed_mask_volume


        return image_volume , mask_volume
        
        

        
        return x, y, z

In [247]:
train_transform = A.Compose(
    [
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

In [251]:
os.path.join(images, os.listdir(images)[index])

'C:\\Users\\Rishabh\\Downloads\\label_192\\label_192\\images\\ID_0b10cbee_ID_f91d6a7cd2.nii.gz'

In [252]:
os.path.join(ground_truths, os.listdir(images)[index])

'C:\\Users\\Rishabh\\Downloads\\label_192\\label_192\\ground truths\\ID_0b10cbee_ID_f91d6a7cd2.nii.gz'

In [248]:
data = Single_Volume_patch_Class_3D(os.path.join(images, os.listdir(images)[index]) , os.path.join(ground_truths, os.listdir(images)[index]), transform=train_transform)

In [249]:
train_loader = DataLoader(
    data,
    batch_size=4,
    num_workers=0,
    pin_memory=True,
    shuffle=True,
)

In [250]:
for batch_idx, (inputs, targets) in enumerate(train_loader):
    print(inputs.shape, ':-', targets.shape)

torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64])
torch.Size([4, 16, 64, 64]) :- torch.Size([4, 16, 64, 64

KeyboardInterrupt: 

In [235]:
len(data)

2523

In [236]:
image_volume , mask_volume = data[2524]

In [237]:
image_volume.shape, mask_volume.shape

(torch.Size([16, 64, 64]), torch.Size([16, 64, 64]))

In [238]:
np.unique(np.array(mask_volume))

array([0.])

In [159]:
x, y, z = data[32]

In [160]:
x, y, z

(48, 16, 0)

In [240]:
for i in range(len(data)):
    image_volume , mask_volume = data[i]
    print(i, ':-' ,np.unique(np.array(mask_volume)))
    # print(x,' ',y,' ',z)

0 :- [0.]
1 :- [0.]
2 :- [0.]
3 :- [0.]
4 :- [0.]
5 :- [0.]
6 :- [0.]
7 :- [0.]
8 :- [0.]
9 :- [0.]
10 :- [0.]
11 :- [0.]
12 :- [0.]
13 :- [0.]
14 :- [0.]
15 :- [0.]
16 :- [0.]
17 :- [0.]
18 :- [0.]
19 :- [0.]
20 :- [0.]
21 :- [0.]
22 :- [0.]
23 :- [0.]
24 :- [0.]
25 :- [0.]
26 :- [0.]
27 :- [0.]
28 :- [0.]
29 :- [0.]
30 :- [0.]
31 :- [0.]
32 :- [0.]
33 :- [0.]
34 :- [0.]
35 :- [0.]
36 :- [0.]
37 :- [0.]
38 :- [0.]
39 :- [0.]
40 :- [0.]
41 :- [0.]
42 :- [0.]
43 :- [0.]
44 :- [0.]
45 :- [0.]
46 :- [0.]
47 :- [0.]
48 :- [0.]
49 :- [0.]
50 :- [0.]
51 :- [0.]
52 :- [0.]
53 :- [0.]
54 :- [0.]
55 :- [0.]
56 :- [0.]
57 :- [0.]
58 :- [0.]
59 :- [0.]
60 :- [0.]
61 :- [0.]
62 :- [0.]
63 :- [0.]
64 :- [0.]
65 :- [0.]
66 :- [0.]
67 :- [0.]
68 :- [0.]
69 :- [0.]
70 :- [0.]
71 :- [0.]
72 :- [0.]
73 :- [0. 1.]
74 :- [0. 1.]
75 :- [0. 1.]
76 :- [0. 1.]
77 :- [0. 1.]
78 :- [0. 1.]
79 :- [0.]
80 :- [0.]
81 :- [0.]
82 :- [0.]
83 :- [0.]
84 :- [0.]
85 :- [0.]
86 :- [0.]
87 :- [0.]
88 :- [0.]
89 :- [0.]
90

KeyboardInterrupt: 

In [241]:
from PIL import Image
def save_prediction(image, mask, name):
    gray_image = image
    binary_mask = mask
    
    gray_norm = gray_image / 255.0

    # Create an RGB image with grayscale as background
    overlay = np.stack([gray_norm, gray_norm, gray_norm], axis=-1)
    
    # Define lighter colors for each class (0-8)
    colors = {
        1: [1.0, 0.6, 0.6],   # Light Red
        2: [0.6, 1.0, 0.6],   # Light Green
        3: [0.6, 0.6, 1.0],   # Light Blue
        4: [1.0, 1.0, 0.6],   # Light Yellow
        5: [1.0, 0.6, 1.0],   # Light Magenta
        6: [0.6, 1.0, 1.0],   # Light Cyan
        7: [0.8, 0.7, 1.0],   # Light Purple
        8: [1.0, 0.8, 0.6]    # Light Orange
    }
    
    # Create an RGB mask initialized with zeros
    mask_rgb = np.zeros_like(overlay)

    
    # Assign colors based on binary_mask values
    for value, color in colors.items():
        mask_rgb[binary_mask == value] = color
    
    # Define transparency level
    alpha = 0.4  # Transparency level (0-1)
    
    # Blend grayscale image with the colored mask
    blended = overlay * (1 - alpha) + mask_rgb * alpha
    blended = (blended*255).astype(np.uint8)
    image = Image.fromarray(blended)
    index = len(os.listdir('Predictions'))+1
    image.save(f'Predictions/output_{name}_{index}.jpg', quality=100)

In [245]:
for i in range(len(data)):
    image_volume , mask_volume = data[i]
    image_volume , mask_volume = np.array(image_volume)*255, np.array(mask_volume)
    if len(np.unique(mask_volume)) > 1:
        for j in range(image_volume.shape[0]):
            if len(np.unique(mask_volume[j,:,:])) > 1:
                save_prediction(image_volume[j,:,:], mask_volume[j,:,:], str(i))
            

KeyboardInterrupt: 